In [88]:
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import chain
from collections import defaultdict

In [89]:
def conjuntos_sobre_umbral_sets(tuplas, conjuntos, frec_min):
    '''
    tuplas: lista de conjuntos de varaibles (tuplas de la base de datos)
    conjuntos: lista con conjuntos de variables
    frec_min: int, umbral minimo para seguir en la proxima iteración 
    '''
    nuevas_tuplas = []
    sobrevivientes = []
    muertes = []
    
    diccionario = defaultdict(int)
    posiciones = defaultdict(list)
    
    #t = datetime.now()
    '''for conjunto in conjuntos:
        for tupla in tuplas:
            se_queda = False
            if conjunto.issubset(tupla):
                diccionario[tuple(conjunto)] += 1
                posiciones[tupla].append(conjunto)
                se_queda = True
                #if tupla not in nuevas_tuplas:
                 #   nuevas_tuplas.append(tupla)'''
    for tupla in tuplas:
        se_queda = False
        for conjunto in conjuntos:
            if conjunto.issubset(tupla):
                diccionario[tuple(conjunto)] += 1
                posiciones[tupla].append(conjunto)
                se_queda = True
        if se_queda:
            nuevas_tuplas.append(tupla)
            

    s = filter(lambda x: x[1] >= frec_min, diccionario.items())
    m = filter(lambda x: x[1] < frec_min, diccionario.items())
    
    sobrevivientes = [{*x[0]} for x in s]
    muertes = [{*x[0]} for x in m]
    
    #print('1.- for de sobre umbral tardo {}\n'.format((datetime.now() - t).total_seconds()))            
    #print('murio:', conjunto)
    #print('\nMe demoré {} seg en el sobre umbral'.format((datetime.now() - t).total_seconds()))
    #print()
    return nuevas_tuplas, sobrevivientes, muertes
    

In [90]:
def join_conjunto_sets2(conjuntos, muertes):
    aux = []
    nuevo_conjunto = []
    siguiente_conjunto = 1
    tiempo = datetime.now()
    
    #t = datetime.now()
    for A in conjuntos[:-1]:
        for B in conjuntos[siguiente_conjunto:]:
            if A - {max(A)} == B - {max(B)}:
                posible = A.union(B)
                muere = False
                aux.append(posible)
                for muerte in muertes:
                    if muerte.issubset(posible):
                        muere = True
                        break
                if not muere:
                    nuevo_conjunto.append(posible)
            else:
                continue

                #if not len(list(filter(lambda muerte: all(dato in C for dato in muerte), muertes))):
                 #   nuevo_conjunto.append(C) 
        siguiente_conjunto += 1
        
    #print('2.- for de sobre umbral tardo {}\n'.format((datetime.now() - t).total_seconds()))

    #print('\nMe demoré {} seg en el join'.format((datetime.now() - t).total_seconds()))
    #print('Join entre \n{}\nCon muertes: {}\ngenero:\n{} \n\n'.format(conjuntos,muertes, nuevo_conjunto))         
    
    return nuevo_conjunto

In [94]:
def fit(frec_min, datos='spotify.npy'):
    tiempo = datetime.now()
    data = np.load(datos)
    listas_datos = [set(x) for x in dict(data.item()).values()]
    frec_min = frec_min * len(listas_datos)
    
    canciones = [x for songs in listas_datos for x in songs]
    orden = list(set(canciones))

    enumeracion_canciones = dict(enumerate(orden))  # {0: cancion1, 1: cancion2}...
    canciones_enumeracion = {x[1]: x[0] for x in enumeracion_canciones.items()}  # {cancion1: 0, cancion2: 1... }
    
    tuplas = list(map(lambda dato: tuple(sorted(list(canciones_enumeracion[x] for x in dato))), listas_datos))

    conjuntos = [{x} for x in canciones_enumeracion.values()]
    #conjuntos = ([x] for x in canciones_enumeracion.values())

    eliminados = []
    retorno = []

    # primer conjunto    
    diccionario = defaultdict(int)
    diccionario = defaultdict(int)

    total = chain(*tuplas)
    for x in total:
        diccionario[x] += 1
    
    conjuntos = [{x[0]} for x in (filter(lambda x: x[1] >= frec_min, diccionario.items()))]
    
    tuplas2 = []
    nueva_tupla = []
    for tupla in tuplas:
        aux = []
        listo = False
        for conjunto in conjuntos:
            if conjunto.issubset(tupla):
                aux.append(*conjunto)
                if not listo:### 
                    listo = True
                    tuplas2.append(tupla)
                #break
        if aux: 
            nueva_tupla.append(tuple(aux))
    
    tuplas = tuplas2

    tiempo = datetime.now()
    while True:
        k += 1
        t = datetime.now()
        tuplas, listas, muerte = conjuntos_sobre_umbral_sets(tuplas, conjuntos, frec_min)

        t = datetime.now()
        conjuntos = join_conjunto_sets2(listas, muerte)

        if not listas:  
            return retorno
       

        else:
            aux = []
            for conjunto in listas:
                l = []
                for dato in conjunto:
                    l.append(enumeracion_canciones[dato])
                aux.append(l)

            retorno.extend(aux)
        tiempo = datetime.now()



In [97]:
t = 0
#for x in range(1000):
tiempo = datetime.now()
a = fit(0.1)

t += (datetime.now() - tiempo).total_seconds()
print('tiempo promedio:', t)

<class 'list'> <class 'tuple'> <class 'int'>
<class 'list'> <class 'tuple'> <class 'int'>
tiempo promedio: 0.000558


In [98]:
for x in a:
    print(x)

[1]
[2]
[5]
[4]
[3]
[1, 2]
[1, 5]
[2, 5]
[2, 4]
[2, 3]
[1, 3]
[1, 2, 5]
[1, 2, 3]


In [84]:
a1 = [['Closer'], ['Forever'], ['Say My Name'], ['Home'], ["Don't Stop Believin'"], ['No Role Modelz'], ['Ignition - Remix'], ['Lean On (feat. MØ & DJ Snake)'], ['Stronger'], ['Riptide'], ['Mercy'], ['One Dance'], ['My Girl'], ['Redbone'], ['Body Like A Back Road'], ['Despacito - Remix'], ["I'm the One"], ['Hello'], ['Ni**as In Paris'], ['Paris'], ['Jumpman'], ['Unforgettable'], ['XO TOUR Llif3'], ['goosebumps'], ['White Iverson'], ['Black Beatles'], ['Bounce Back'], ['Congratulations'], ['Let It Go'], ['Hallelujah'], ['Blessings'], ['Skinny Love'], ['Fake Love'], ['Caroline'], ['Shape of You'], ["That's What I Like"], ['Slide'], ['Say It'], ['Alright'], ['T-Shirt'], ['Starboy'], ['September'], ['Gold'], ['Down'], ['Let Me Love You'], ['Stay'], ["Don't"], ['1-800-273-8255'], ['Location'], ['Hurricane'], ['Roses'], ['Sorry'], ['Ride'], ['Trap Queen'], ['HUMBLE.'], ['Passionfruit'], ["Don't Let Me Down"], ['Broccoli (feat. Lil Yachty)'], ['Panda'], ['Mask Off'], ['Bad and Boujee (feat. Lil Uzi Vert)'], ['No Problem (feat. Lil Wayne & 2 Chainz)'], ['Antidote'], ['Work'], ['iSpy (feat. Lil Yachty)'], ['Hotline Bling'], ['The Hills'], ['Gold Digger']]
a2 = [['Forever'], ['Say My Name'], ['Closer'], ['Home'], ['Lean On (feat. MØ & DJ Snake)'], ['Stronger'], ["Don't Stop Believin'"], ['Ignition - Remix'], ['No Role Modelz'], ['Riptide'], ['Mercy'], ['One Dance'], ['Redbone'], ['My Girl'], ['Body Like A Back Road'], ["I'm the One"], ['Despacito - Remix'], ['Hello'], ['Bounce Back'], ['goosebumps'], ['Unforgettable'], ['XO TOUR Llif3'], ['White Iverson'], ['Black Beatles'], ['Jumpman'], ['Ni**as In Paris'], ['Congratulations'], ['Paris'], ['Let It Go'], ['Hallelujah'], ['Blessings'], ['Skinny Love'], ['Fake Love'], ['Caroline'], ["That's What I Like"], ['Shape of You'], ['Say It'], ['Starboy'], ['Slide'], ['Alright'], ['T-Shirt'], ['September'], ['Gold'], ['Down'], ['Let Me Love You'], ['Stay'], ['Location'], ['1-800-273-8255'], ["Don't"], ['Sorry'], ['Hurricane'], ['Roses'], ['Ride'], ['Trap Queen'], ['HUMBLE.'], ['Passionfruit'], ["Don't Let Me Down"], ['Panda'], ['Broccoli (feat. Lil Yachty)'], ['Bad and Boujee (feat. Lil Uzi Vert)'], ['Mask Off'], ['No Problem (feat. Lil Wayne & 2 Chainz)'], ['Antidote'], ['Work'], ['iSpy (feat. Lil Yachty)'], ['Hotline Bling'], ['The Hills'], ['Gold Digger']]

In [87]:
n = 0
for x in a1:
    if x not in a2:
        print(x,' No esta')
    else:
        n += 1
        
print(n == len(a1) == len(a2))

True
